# Symbolic Regression

## Imports

In [202]:
import numpy as np
import random
from copy import deepcopy
import pandas as pd
import plotly.graph_objects as go
from random import choice
import tqdm
from sklearn.model_selection import KFold

np.seterr(all='ignore')  # ignore all numpy warnings, 
None

## Operators and Parameters Configuration

In [203]:
# Define the function set
OPERATORS = [
    {"func": np.add, "arity": 2, "format_str": "({} + {})"},
    {"func": np.subtract, "arity": 2, "format_str": "({} - {})"},
    {"func": np.multiply, "arity": 2, "format_str": "({} * {})"},
    {"func": np.sin, "arity": 1, "format_str": "sin({})"},
    {"func": np.cos, "arity": 1, "format_str": "cos({})"},
    {"func": np.exp, "arity": 1, "format_str": "exp({})"},  
    {"func": np.abs, "arity": 1, "format_str": "abs({})"},
    {"func": np.arctan, "arity": 1, "format_str": "arctan({})"},
    {"func": np.cosh, "arity": 1, "format_str": "cosh({})"},
    {"func": np.sinh, "arity": 1, "format_str": "sinh({})"},
    {"func": np.tanh, "arity": 1, "format_str": "tanh({})"},
]
None

In [204]:
# Parameters
MAX_DEPTH = 6
POP_SIZE = 300
TOURNAMENT_SIZE = 5
CROSSOVER_RATE = 0.50
MAX_GENERATIONS = 1500
ERC_RATE = 0.05
MUTATION_TREE_RATE = 0.2
DYNAMIC_EXPLORATION = False

## Select input dataset

In [ ]:
file_path = "../data/problem_3.npz" # change problem_N.npz to test different problems N = (1, 2, 3, 4, 5, 6, 7, 8)

## Load input dataset

In [205]:
# Load the dataset
def load_npz_dataset(file_path):
    data = np.load(file_path)
    features = data['x'].T
    target = data['y']
    return features, target


## Metric: Mean Square Error

In [206]:

def evaluate_program(node, data):
    if node["type"] == "x":
        return data[node["name"]]
    if node["type"] == "const":
        return np.full(len(next(iter(data.values()))), node["value"])
    # Evaluate children
    children_values = [evaluate_program(child, data) for child in node["children"]]
    if any(child is None for child in children_values):
        return None  # discard invalid programs

    try:
        result = node["operator"]["func"](*children_values)
        if np.any(np.isinf(result)) or np.any(np.isnan(result)):
            return None  # discard invalid programs
        result = np.clip(result, -1e10, 1e10)  # Limit the output range
        return result
    except (FloatingPointError, ZeroDivisionError, ValueError):
        return None  # discard invalid programs


def MSE(program, data, target):
    """
    Calculates the MSE of a program. If the program is valid, it computes the mean squared error (MSE).
    If the program is invalid (None, NaN, Inf), it will return a high value to exclude it from the evolutionary process.
    """

    result = evaluate_program(program, data)
    if result is None:
        return float("inf")  # Penalize invalid programs
    try:
        # claculate the mean squared error (MSE) as a measure of fitness
        error = np.mean((result - target) ** 2)
        return error
    except ValueError:  # If there is an error during the calculation of the MSE, penalize the program
        return float("inf")
None


## Initialization

In [207]:
def grow(max_depth, features):
    if max_depth == 0 or random.random() > 0.5:
        if random.random() < 0.99:
            return {"type": "x", "name": random.choice(features)}  # Variabile
        else:
            return {"type": "const", "value": random.choice([x for x in range(-10, 11) if x != 0])}  # Costante
    operator = random.choice(OPERATORS)
    children = [grow(max_depth - 1, features) for _ in range(operator["arity"])]
    return {"type": "op", "operator": operator, "children": children}

def full(max_depth, features):

    if max_depth == 0:
        if random.random() < 0.99:
            return {"type": "x", "name": random.choice(features)}
        else:
            return {"type": "const", "value": random.choice([x for x in range(-10, 11) if x != 0])}
    operator = random.choice(OPERATORS)
    children = [full(max_depth - 1, features) for _ in range(operator["arity"])]
    return {"type": "op", "operator": operator, "children": children}

def half_and_half(max_depth, features):
    if random.random() < 0.5:
        return grow(max_depth, features)
    else:
        return full(max_depth, features)



## Global Helper Function

In [208]:
def render_program(node):
    if node["type"] == "x":
        return node["name"]
    if node["type"] == "const":
        return f"{node['value']:.2f}"
    children = [render_program(child) for child in node["children"]]
    return node["operator"]["format_str"].format(*children)

## Selection

In [209]:

def tournament_selection(population, MSE_scores):
    selected = random.choices(population, k=TOURNAMENT_SIZE)
    return min(selected, key=lambda p: MSE_scores[p])
None

## Validation

In [210]:
# verify if the tree is valid
def validate_tree(node):
    if node["type"] == "op":
        if len(node["children"]) != node["operator"]["arity"]:
            return False
        return all(validate_tree(child) for child in node["children"])
    return True  # leaf nodes are always valid
def contains_all_vars(program, features):
    # verify if the program contains all the variables in the dataset
    required_vars = set(features)
    found_vars = set()

    def traverse(node):
        if node["type"] == "x":
            found_vars.add(node["name"])

        # if the node is an operator, traverse its children
        if node["type"] == "op":
            for child in node["children"]:
                traverse(child)

    traverse(program)
    
    # verify if the program contains all the required variables
    return required_vars.issubset(found_vars)


## XOver Helper Functions

In [211]:
def get_leaf_nodes(node):
    """
    Retrieves all leaf nodes given a parent node.
    """
    # If the node has no children, it is a leaf
    if "children" not in node or not node["children"]:
        return [node]

    # Otherwise, recursively get the leaf nodes of the children
    leaf_nodes = []
    for child in node["children"]:
        leaf_nodes.extend(get_leaf_nodes(child))

    return leaf_nodes


def replace_leaf(node, old_leaf, new_leaf):
    """
    Recursively replaces a leaf in a tree.
    """
    # if the current node is the old leaf, replace it with the new leaf
    if node == old_leaf:
        return deepcopy(new_leaf)
    # If the current node has children, explore recursively.
    if "children" in node:
        node["children"] = [replace_leaf(child, old_leaf, new_leaf) for child in node["children"]]

    return node

def find_parent(node, target):
    """
    Finds the parent of a target node in a tree.
    """

    if "children" not in node or not node["children"]:
        return None

    for child in node["children"]:
        if child is target:
            return node
        result = find_parent(child, target)
        if result:
            return result

    return None

def get_operator_nodes(node):
    """
    Returns all operator nodes (non-leaf) of a tree.
    """
    nodes = []
    if node.get("type") == "op":
        nodes.append(node)
    if "children" in node:
        for child in node["children"]:
            nodes.extend(get_operator_nodes(child))
    return nodes


def get_subtree_height(node):
    """
    Calculates the height of a subtree from the leaves towards the given node.
    """

    if "children" not in node or not node["children"]:
        # If the node has no children, it is a leaf and has height 0
        return 0
    return 1 + max(get_subtree_height(child) for child in node["children"])


def select_similar_height_node(node, target_height):
    """
    Selects a node in a tree with a height similar to the target_height.
    """

    operator_nodes = get_operator_nodes(node)
    best_match = None
    min_diff = float("inf")
    for op_node in operator_nodes:
        height = get_subtree_height(op_node)
        diff = abs(height - target_height)
        if diff < min_diff:
            min_diff = diff
            best_match = op_node
    return best_match

# Function to replace a subtree in a tree
def replace_subtree(parent, target, replacement):
    if parent == target:
        return deepcopy(replacement)
    if "children" in parent:
        parent["children"] = [
            replace_subtree(child, target, replacement) for child in parent["children"]
        ]
    return parent


## XOver Methods

In [212]:
#Crossover where a random crossover point is chosen and the subtrees are swapped.
def crossover_random_height(parent1, parent2):
    # create deep copies of the parents to avoid unwanted modifications
    new_parent1 = deepcopy(parent1)
    new_parent2 = deepcopy(parent2)

    # Verify that both parents are operator nodes with valid children
    if new_parent1["type"] != "op" or new_parent2["type"] != "op":
        return new_parent1  # No crossover possible, return the first parent
    # Verify that both parents have children
    if len(new_parent1["children"]) == 0 or len(new_parent2["children"]) == 0:
        return new_parent1  # No crossover possible, return the first parent

    # 90% swap non-leaf nodes
    if random.random() < 0.9:

        if len(new_parent1["children"]) != new_parent1["operator"]["arity"] or len(new_parent2["children"]) != new_parent2["operator"]["arity"]:
            return new_parent1  # No crossover possible, return the first parent
        # Select a random crossover point in parent1 and parent2
        swap_point1 = random.randint(0, len(new_parent1["children"]) - 1)
        swap_point2 = random.randint(0, len(new_parent2["children"]) - 1)

        # Swap the subtrees
        new_parent1["children"][swap_point1], new_parent2["children"][swap_point2] = (
            deepcopy(new_parent2["children"][swap_point2]),
            deepcopy(new_parent1["children"][swap_point1]),
        )
    else:
        # 10% swap leaf nodes
        leaves1 = get_leaf_nodes(new_parent1)
        leaves2 = get_leaf_nodes(new_parent2)

        if not leaves1 or not leaves2:
            return new_parent1  # No crossover possible if there are no leaves

        # Select a random leaf from each parent
        leaf1 = choice(leaves1)
        leaf2 = choice(leaves2)

        # Replace the leaf
        new_parent1 = replace_leaf(new_parent1, leaf1, leaf2)
        new_parent2 = replace_leaf(new_parent2, leaf2, leaf1)

    return new_parent1


"""
    Crossover with random (similar) height, 90% swaps non-leaf nodes, 10% swaps leaf nodes.
"""
def crossover_similar_height(parent1, parent2):
    # Create deep copies of the parents to avoid unwanted modifications
    new_parent1 = deepcopy(parent1)
    new_parent2 = deepcopy(parent2)

    # Verify that both parents are operator nodes with valid children
    if new_parent1["type"] != "op" or new_parent2["type"] != "op":
        return new_parent1  # No crossover possible, return the first parent
    if len(new_parent1["children"]) == 0 or len(new_parent2["children"]) == 0:
        return new_parent1  # No crossover possible, return the first parent
    if random.random() < 0.9:
        # Selects a random swap point in parent1.
        operator_nodes1 = get_operator_nodes(new_parent1)
        if not operator_nodes1:
            return new_parent1  # No valid swap point in parent1
        swap_point1 = random.choice(operator_nodes1)

        # Get the height of the subtree
        height1 = get_subtree_height(swap_point1)

        # Select a similar height node in parent2
        swap_point2 = select_similar_height_node(new_parent2, height1)
        if not swap_point2:
            return new_parent1  # No valid swap point in parent2

        new_parent1 = replace_subtree(new_parent1, swap_point1, swap_point2)
        new_parent2 = replace_subtree(new_parent2, swap_point2, swap_point1)
    else:
        # 10% swap leaf nodes        
        leaves1 = get_leaf_nodes(new_parent1)
        leaves2 = get_leaf_nodes(new_parent2)

        if not leaves1 or not leaves2:
            return new_parent1  # No crossover possible if there are no leaves
        
        # Select a random leaf from each parent
        leaf1 = choice(leaves1)
        leaf2 = choice(leaves2)

        # Replace the leaf
        new_parent1 = replace_leaf(new_parent1, leaf1, leaf2)
        new_parent2 = replace_leaf(new_parent2, leaf2, leaf1)

    return new_parent1

    


def crossover_fixed_height(parent1, parent2):
    """
    Perform a crossover between two trees, swapping subtrees with a maximum height of 3.
    """
    # Create deep copies of the parents to avoid unwanted modifications
    new_parent1 = deepcopy(parent1)
    new_parent2 = deepcopy(parent2)

    # Verify that both parents are operator nodes with valid children
    if new_parent1["type"] != "op" or new_parent2["type"] != "op":
        return new_parent1  # No crossover possible, return the first parent
    if len(new_parent1["children"]) == 0 or len(new_parent2["children"]) == 0:
        return new_parent1  # No crossover possible, return the first parent
    if random.random() < 0.9:
        # Selects a random swap point in parent1 with a maximum height of 3
        operator_nodes1 = [node for node in get_operator_nodes(new_parent1)
                        if get_subtree_height(node) <= 2]
        operator_nodes2 = [node for node in get_operator_nodes(new_parent2)
                        if get_subtree_height(node) <= 2]

        if not operator_nodes1 or not operator_nodes2:
            return new_parent1  # No valid swap point in parent1

        # Select a random swap point in parent1 and parent2
        swap_point1 = random.choice(operator_nodes1)
        swap_point2 = random.choice(operator_nodes2)

        new_parent1 = replace_subtree(new_parent1, swap_point1, swap_point2)
        new_parent2 = replace_subtree(new_parent2, swap_point2, swap_point1)
    else:
        leaves1 = get_leaf_nodes(new_parent1)
        leaves2 = get_leaf_nodes(new_parent2)

        if not leaves1 or not leaves2:
            return new_parent1  # No crossover possible if there are no leaves
        # Select a random leaf from each parent
        leaf1 = choice(leaves1)
        leaf2 = choice(leaves2)

        # Replace the leaf
        new_parent1 = replace_leaf(new_parent1, leaf1, leaf2)
        new_parent2 = replace_leaf(new_parent2, leaf2, leaf1)

    return new_parent1


## Mutation Helper Functions

In [213]:
def get_subtrees_with_height(node, parent=None, index=None, current_height=0):
        """
        Retrieves all subtrees (pointers to nodes, parents, indices, and heights).
        """
        subtrees = []
        if node["type"] == "op":  # only consider operator nodes
            for i, child in enumerate(node["children"]):
                subtree_height = get_subtree_height(child)
                subtrees.append((node, i, subtree_height))  # Store the parent, index, and height
                subtrees.extend(get_subtrees_with_height(child, node, i, current_height + 1))
        return subtrees

def get_subtree_height(node):
    """
    Calculates the height of a subtree from the leaves towards the given node.
    """
    if node["type"] != "op" or not node.get("children"):
        return 0  # Leaf nodes have height 0
    return 1 + max(get_subtree_height(child) for child in node["children"])

def perform_erc_mutation():
    """
    Perform the ERC mutation.
    """
    while True:
        value = random.uniform(-4, 4)  # Generate a random value
        if value != 0:  # Ensure the value is not zero
            return {"type": "const", "value": value}




## Mutation Methods

In [214]:
def mutate_all_tree(features, max_depth):
    return half_and_half(max_depth, features)

def mutate(program, features, max_depth):
    # Create a deep copy of the program to avoid unwanted modifications
    program = deepcopy(program)
    if program["type"] == "op":
        # if the node is an operator, randomly change the operator
        program["operator"] = random.choice(OPERATORS)
        # Recursively mutate the children
        child_idx = random.randint(0, len(program["children"]) - 1)
        program["children"][child_idx] = mutate(program["children"][child_idx], features, max_depth - 1)
    
    elif program["type"] == "x":
        # if the node is a leaf, randomly change the variable
        program["value"] = random.choice(features)  # Randomly select a new variable

    return program


def mutate_erc(program, features, max_depth):
    """
    Perform the ERC mutation.
    """
    # Create a deep copy of the program to avoid unwanted modifications
    program = deepcopy(program)
    if program["type"] != "op":
            # If the current node is a leaf, perform the ERC mutation
        return perform_erc_mutation()

    # If the current node is an operator, recursively mutate the children
    if program["type"] == "op":
        child_idx = random.randint(0, len(program["children"]) - 1)
        program["children"][child_idx] = mutate_erc(program["children"][child_idx], features, max_depth - 1)

    return program

def swap_mutation(program):
    # Create a deep copy of the program to avoid unwanted modifications
    program = deepcopy(program)
    # Get all subtrees with their heights
    subtrees = get_subtrees_with_height(program)
    if len(subtrees) < 2:
        return program  # No mutation possible
    # Select a random subtree
    parent1, idx1, height1 = random.choice(subtrees)

    subtrees_sorted_by_height_diff = sorted(
        subtrees,
        key=lambda x: abs(x[2] - height1)  # Sort by the absolute difference in height
    )

   # filter out subtrees that are too similar in height
    for parent2, idx2, _ in subtrees_sorted_by_height_diff:
        if (parent1, idx1) != (parent2, idx2):  # Ensure the subtrees are different
            # Swap the subtrees
            parent1["children"][idx1], parent2["children"][idx2] = (
                deepcopy(parent2["children"][idx2]),
                deepcopy(parent1["children"][idx1]),
            )
            break

    return program

## Evolution

In [215]:
mean_MSE = []
# Function to dynamically change the exploration rate

def dynamic_change(MSE_scores):
    """
    Dynamically adjust the exploration rate based on changes in the MSE (Mean Squared Error).

    Parameters:
        MSE_scores (dict): Dictionary containing the MSE scores.

    Returns:
        float: Updated exploration rate.
    """
    # Calculate and store the mean of the current MSE scores
    mean_MSE.append(np.mean(list(MSE_scores.values())))

    # Default exploration rate
    exploration_rate = 0.7

    # Adjust exploration rate if we have more than 100 iterations
    if len(mean_MSE) > 101:
        current_MSE = mean_MSE[-1]
        previous_MSE = mean_MSE[-99]  # Compare with MSE from 99 steps ago

        # Calculate percentage change in the MSE
        change_percentage = (current_MSE - previous_MSE) / previous_MSE if previous_MSE != 0 else 0

        # Adjust exploration rate based on the change in the MSE
        if change_percentage < 0:
            # Reduce exploration rate if the MSE has improved (decreased)
            exploration_rate = max(0.5, 1 + change_percentage)
        else:
            # Increase exploration rate if the MSE has worsened or stayed the same
            exploration_rate = 0.5

    return exploration_rate



In [216]:

def evolve(features, data, target):
    # Inizialize the population with valid individuals only (i.e., those that contain all the variables)
    population = []
    while len(population) < POP_SIZE:
        candidate = half_and_half(MAX_DEPTH, features)
        if evaluate_program(candidate, data) is not None:  # Ensure the program is valid
            population.append(candidate)

    best_program = None
    best_MSE = float("inf")
    # Evolve the population for a set number of generations
    for _ in tqdm.tqdm(range(MAX_GENERATIONS)):
        # calculate the MSE scores for the population
        MSE_scores = {}
        for prog in population:
            prog_repr = render_program(prog)
            score = MSE(prog, data, target)
            MSE_scores[prog_repr] = score
        
        # Map the programs to their string representations
        program_mapping = {render_program(prog): prog for prog in population}
        new_population = []
        if DYNAMIC_EXPLORATION:
            exploration_rate = dynamic_change(MSE_scores)
        else:
            exploration_rate = 0.5
        # Generate new individuals for the population
        while len(new_population) < POP_SIZE:
            valid_flag = False
            while not valid_flag:
                parent1_repr = tournament_selection(list(MSE_scores.keys()), MSE_scores)
                parent1 = program_mapping[parent1_repr]
                if random.random() < CROSSOVER_RATE:
                    parent2_repr = tournament_selection(list(MSE_scores.keys()), MSE_scores)
                    parent2 = program_mapping[parent2_repr]
                    # choose between crossover_similar_height and crossover_2
                    crossover_probability = random.random()
                    #exploration
                    if crossover_probability < exploration_rate:
                        if random.random() < 0.5:
                            offspring = crossover_random_height(parent1, parent2)
                        else:
                           offspring = crossover_similar_height(parent1, parent2)
                    else:
                        #exploitation
                        offspring = crossover_fixed_height(parent1, parent2)
                else:
                    random_number = random.random()
                    if random_number <= ERC_RATE:
                        offspring = mutate_erc(parent1, features, MAX_DEPTH)
                    elif random_number > ERC_RATE and random.random() < MUTATION_TREE_RATE:
                        offspring = mutate_all_tree(features, MAX_DEPTH)
                    else:
                        # choose between swap_mutation and mutate
                        if random.random() < 0.9:
                            offspring = swap_mutation(parent1)
                        else:
                            offspring = mutate(parent1, features, MAX_DEPTH)

                # Validate the offspring
                if validate_tree(offspring) and evaluate_program(offspring, data) is not None and contains_all_vars(offspring, features):
                    new_population.append(offspring)
                    valid_flag = True
        
        # Combine the old and new populations
        combined_population =  new_population + population
        # Calculate the MSE scores for the combined population
        MSE_scores = {render_program(prog): MSE(prog, data, target) for prog in combined_population}
        # Sort the combined population by their MSE scores
        sorted_population = sorted(combined_population, key=lambda prog: MSE_scores[render_program(prog)])
        
        # Select the top 300 individuals
        population = sorted_population[:300]
        # Update the best program and its MSE score
        
        for prog in population:
            prog_repr = render_program(prog)
            score = MSE_scores[prog_repr]
            if score < best_MSE:
                best_MSE = score
                best_program = prog
        if best_MSE == 0:
            return best_program, best_MSE
    
    return best_program, best_MSE


## Plots

In [217]:


def plot_results_3D(best_program, features, target):
    X1 = features[:, 0]
    X2 = features[:, 1]

    x1_grid = np.linspace(X1.min(), X1.max(), 50)
    x2_grid = np.linspace(X2.min(), X2.max(), 50)
    x1_mesh, x2_mesh = np.meshgrid(x1_grid, x2_grid)

    data_range = {
        "x0": x1_mesh.ravel(),
        "x1": x2_mesh.ravel(),
    }

    y_pred_flat = evaluate_program(best_program, data_range)
    y_pred_mesh = y_pred_flat.reshape(x1_mesh.shape)

    fig = go.Figure()

 
    fig.add_trace(go.Scatter3d(
        x=X1,
        y=X2,
        z=target,
        mode='markers',
        marker=dict(size=4, color='blue'),
        name="Real Data"
    ))


    fig.add_trace(go.Surface(
        z=y_pred_mesh,
        x=x1_grid,
        y=x2_grid,
        colorscale='Reds',
        opacity=0.7,
        name="Approximation"
    ))

 
    fig.update_layout(
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='y'
        ),
        title="Comparing the Real and Predicted Data",
        autosize=True
    )


    fig.show()
None

## K-FOLD validation

In [218]:

# function to perform k-fold cross validation
def k_fold_cross_validation(features, target, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_scores = []
    best__MSE = float("inf")
    best_p = None
    feature_names = [f"x{i}" for i in range(features.shape[1])]
    fold=0
    for train_idx, val_idx in kf.split(features):
        valid_flag = False
        while not valid_flag:
            train_features, val_features = features[train_idx], features[val_idx]
            train_target, val_target = target[train_idx], target[val_idx]
            # Create a dictionary with the training and validation data
            train_data_dict = {f"x{i}": train_features[:, i] for i in range(train_features.shape[1])}
            val_data_dict = {f"x{i}": val_features[:, i] for i in range(val_features.shape[1])}
            
            # Evolve the best program for the current fold
            best_program, _ = evolve(features=feature_names, data=train_data_dict, target=train_target)
            
            if best_program is None:
                print("Fold discarded: invalid program, retrying evolution")
            # Calculate the MSE of the best program on the validation data
            val_MSE = MSE(best_program, val_data_dict, val_target) 
            if val_MSE == float("inf"):
                print("Fold discarded: invalid program, retrying evolution")
                print(f"Program invalid: {render_program(best_program)}")
                # Save the data to a CSV file
                pd.DataFrame(val_data_dict).to_csv("data.csv")
            else: 
                valid_flag = True
                fold_scores.append(val_MSE)
                fold+=1
                # Reset the mean_MSE list
                mean_MSE.clear()
                print(f"Fold {fold}° validated with MSE: {val_MSE:.4f}")
                print(f"Best program of fold {fold}°: {render_program(best_program)}")
                #print("plot of the results on all the dataset: ")
                #plot_results(best_program, features, target)
                if val_MSE < best__MSE:
                    best__MSE = val_MSE
                    best_p = best_program
    
    return best_p, best__MSE



## Main part

In [ ]:

# Load the dataset
features, target = load_npz_dataset(file_path)
if file_path == "../data/problem_8.npz":
    #do a sampling of the dataset taking only 10000 samples (randomly) to speed up the process
    idx = np.random.choice(features.shape[0], 10000, replace=False)
    features_sampled = features[idx]
    target_sampled = target[idx]
    best_p, best_MSE = k_fold_cross_validation(features_sampled, target_sampled, k=5)
else:
    # Perform k-fold cross validation
    best_p, best_MSE = k_fold_cross_validation(features, target, k=5)
print(f"Best program found: {render_program(best_p)} MSE: {best_MSE:.4f} and MSE (all dataset): {MSE(best_p, {f'x{i}': features[:, i] for i in range(features.shape[1])}, target):.4f}")


## Plots

In [19]:
def plot_results_2D(best_program, features, target):
 
    X = features[:, 0]


    x_grid = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)

    data_range = {
        "x0": x_grid.ravel(),
    }


    y_pred_flat = evaluate_program(best_program, data_range)
    y_pred_grid = y_pred_flat.reshape(x_grid.shape)

    fig = go.Figure()

    fig.add_trace(go.Scatter( x=X, y=target, mode='markers', marker=dict(size=4, color='blue'), name="Real Data"))

    fig.add_trace(go.Scatter(x=x_grid.ravel(), y=y_pred_grid.ravel(), mode='lines', line=dict(color='red'), name="Approximation"))


    fig.update_layout(
        xaxis_title='x',
        yaxis_title='y',
        title="comparison between real points and approximated function",
        autosize=True
    )

    fig.show()



In [20]:
if features.shape[1] == 2:
    plot_results_3D(best_p, features, target)

if features.shape[1] == 1:
    plot_results_2D(best_p, features, target)